<a href="https://colab.research.google.com/github/CalvHobbes/fcc-ai/blob/main/makemore_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! python --version

Python 3.10.12
